# Johns Hopkins COVID-19 Dataset in Pandas

COVID-19 is ravaging the globe. Let's look at the excellent Johns Hopkins
dataset using Pandas. This will serve both as a guideline for getting the data
and exploring on your own, as well as an example of Pandas multi-indexing in an
easy to understand situation.

In [ ]:
# Display final value in cell even if assigned
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from urllib.error import HTTPError

plt.style.use("ggplot")

Now that we've made some basic imports, let's write a function that can
read in a datafile from GitHub:

In [ ]:
def get_day(day: pd.Timestamp) -> pd.DataFrame:

    # Read in a datafile from GitHub
    try:
        table = pd.read_csv(
            "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/"
            "master/csse_covid_19_data/csse_covid_19_daily_reports/"
            f"{day:%m-%d-%Y}.csv",
        )
    except HTTPError:
        return pd.DataFrame()

    # Cleanup - sadly, the format has changed a bit over time - we can normalize that here
    table.columns = [
        f.replace("/", "_")
        .replace(" ", "_")
        .replace("Latitude", "Lat")
        .replace("Longitude", "Long_")
        for f in table.columns
    ]

    # This column is new in recent datasets
    if "Admin2" not in table.columns:
        table["Admin2"] = None

    # New datasets have these, but they are not very useful for now
    table.drop(
        columns=["FIPS", "Combined_Key", "Lat", "Long_"], errors="ignore", inplace=True
    )

    # If the last update time was useful, we would make this day only, rather than day + time
    #   table["Last_Update"] = pd.to_datetime(table["Last_Update"]).dt.normalize()
    #
    # However, last update is odd, let's just make this the current day
    table["Last_Update"] = day

    # Make sure indexes are not NaN, which causes later bits to not work. 0 isn't
    # perfect, but good enough.
    # Return as a multindex
    return table.fillna(0).set_index(
        ["Last_Update", "Country_Region", "Province_State", "Admin2"], drop=True
    )

Now let's loop over all days and build a multi-index DataFrame with the whole
dataset. We'll be doing quite a bit of cleanup here as well. If you do this
outside of a function, you should never modify an object in multiple cells;
ideally you create an object like `df`, and make any modifications and
replacements in the same cell. That way, running any cell again or running a
cell multiple times will not cause unusual errors and problems to show up.

In [ ]:
def get_all_days(
    *, start_day: Optional[str] = None, end_day: Optional[str] = None
) -> pd.DataFrame:

    # Make a list of all dates
    date_range = pd.date_range(
        start_day or "2020-01-22", end_day or pd.Timestamp.now().normalize()
    )

    # Create a generator that returns each day's dataframe
    day_gen = (get_day(day) for day in date_range)

    # Make a big dataframe, NaN is 0
    df = pd.concat(day_gen).fillna(0).astype(int)

    # Remove a few duplicate keys
    df = df.groupby(level=df.index.names).sum()

    # Sometimes active is not filled in; we can compute easily
    df["Active"] = np.clip(df["Confirmed"] - df["Deaths"] - df["Recovered"], 0, None)

    # Change in confirmed cases (placed in a pleasing location in the table)
    df.insert(
        1,
        "ΔConfirmed",
        df.groupby(level=("Country_Region", "Province_State", "Admin2"))["Confirmed"]
        .diff()
        .fillna(0)
        .astype(int),
    )

    # Change in deaths
    df.insert(
        3,
        "ΔDeaths",
        df.groupby(level=("Country_Region", "Province_State", "Admin2"))["Deaths"]
        .diff()
        .fillna(0)
        .astype(int),
    )

    return df

If this were a larger/real project, it would be time to bundle up the functions
above and put them into a `.py` file - notebooks are for experimentation,
teaching, and high level manipulation. Functions and classes should normally
move to normal Python files when ready.

Let's look at a few lines of this DataFrame to see what we have:

In [ ]:
%%time
df = get_all_days(end_day="2020-08-01")

The benefit of doing this all at once, in one DataFrame, should quickly become
apparent. We can now use simple selection and grouping to "ask" almost anything
about our dataset.

As an example, let's look at just the US portion of the dataset. We'll use the
pandas selection `.xs`:

In [ ]:
us = df.xs("US", level="Country_Region")

Notice we have counties (early datasets just have one "county" called `"0"`). If
we were only interested in states, we can group by the remaining levels and sum
out the `"Admin2"` (county and similar) dimension:

In [ ]:
by_state = us.groupby(level=("Last_Update", "Province_State")).sum()

Using the same selector as before, we can pick out North Carolina:

In [ ]:
by_state.xs("North Carolina", level="Province_State")

We can look at all of US, as well:

In [ ]:
all_states = by_state.groupby(level="Last_Update").sum()

#### US total cases

Let's try a simple plot first; this is the one you see quite often.

In [ ]:
plt.figure(figsize=(10, 5))
all_states.Confirmed.plot(logy=True, style="o");

#### Italy, new cases per day

As another example, let's view the new cases per day for Italy. We will add a
rolling mean, just to help guide the eye through the fluctuations - it is not a
fit or anything fancy.

In [ ]:
interesting = df.xs("Italy", level="Country_Region").groupby(level="Last_Update").sum()

plt.figure(figsize=(10, 5))
interesting.ΔConfirmed.rolling(5, center=True).mean().plot(
    style="-", label="Rolling mean"
)
interesting.ΔConfirmed.plot(style="o", label="Data")
plt.ylabel("New cases per day")
plt.legend();

#### Italy, transmission rate

It's more interesting to instead look at the transmission rate per day, which is
new cases / active cases. The colors in the plot start changing when Italy
implemented a lockdown on the 11th, and change over 14 days, which is roughly 1x
the time to first symptoms. The lockdown make take longer than that to take full
effect. There were several partial steps taken before the full lockdown on the
4th and 9th. Notice the transmission is slowing noticeably!

In [ ]:
interesting = df.xs("Italy", level="Country_Region").groupby(level="Last_Update").sum()
growth = interesting.ΔConfirmed / interesting.Active
growth = growth["2020-02-24":]

# Color based on lockdown (which happened in 3 stages, 4th, 9th, and 11th)
lockdown = growth.index - pd.Timestamp("2020-03-11")
lockdown = np.clip(lockdown.days, 0, 14) / 14

fix, ax = plt.subplots(figsize=(10, 5))
ax.scatter(growth.index, growth, cmap="cool", c=lockdown)

ax.set_ylabel("new cases / active cases")

# set ticks every week
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
# set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"));

#### US, transmission rate

Same plot for the US. The colors in the plot start changing when the US started
the 15 plan to slow the spread, and change over 14 days, which is roughly 1x the
time to first symptoms. Each state has implemented different guidelines, so the
effect will be spread out even further. Again, we are see the effect of the
lockdown!

In [ ]:
interesting = df.xs("US", level="Country_Region").groupby(level="Last_Update").sum()
growth = interesting.ΔConfirmed / interesting.Active
growth = growth["2020-03-01":]

# Not really a full lockdown, just a distancing guideline + local lockdowns later
lockdown = growth.index - pd.Timestamp("2020-03-15")
lockdown = np.clip(lockdown.days, 0, 14) / 14

fix, ax = plt.subplots(figsize=(10, 5))
ax.scatter(growth.index, growth, cmap="cool", c=lockdown)

ax.set_ylabel("new cases / active cases")

# set ticks every week
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
# set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"));